# Storing Text objects in Postgres

This tutorial demonstrates how to store and query estnltk text objects in postgres database.

In [33]:
from estnltk import Text
from estnltk.storage.postgres import PostgresStorage, JsonbTextQuery, JsonbLayerQuery, create_schema, delete_schema
from estnltk.taggers import VabamorfTagger

The first line of the `pgpass` file that matches the given `PostgresStorage` arguments is used to connect to an existing PostgreSQL database. File format:

    hostname:port:database:username:password

In [34]:
storage = PostgresStorage(pgpass_file='~/.pgpass',
                          dbname='test_db',
                          schema='my_schema')
create_schema(storage)

INFO:postgres_storage.py:97: connecting to host: 'localhost', port: '5432', dbname: 'test_db', user: 'pault'
INFO:postgres_storage.py:109: role: 'pault'


## Collections

Collection stores text objects and provides read/write API.

Create a new collection:

In [35]:
collection = storage.get_collection("my_collection")
collection.create('demo collection')

INFO:db.py:84: new empty collection 'my_collection' created


Add some data:

In [36]:
text1 = Text('ööbik laulab.').tag_layer(['morph_analysis'])
key1 = collection.insert(text1)
print(key1, text1)

text2 = Text('öökull ei laula.').tag_layer(['morph_analysis'])
key2 = collection.insert(text2, key=7)
print(key2, text2)

1 Text(text='ööbik laulab.')
7 Text(text='öökull ei laula.')


In [37]:
collection

AttributeError: 'PostgresStorage' object has no attribute 'count_rows'

Iterate over collection:

In [38]:
for key, text in collection.select():
    print(key, text)

1 Text(text='ööbik laulab.')
7 Text(text='öökull ei laula.')


Search for a particular entry by key:

In [39]:
txt = collection.select_by_key(7)
print(txt)

Text(text='öökull ei laula.')


Search using layer attributes:

In [40]:
q = JsonbTextQuery('morph_analysis', lemma='laulma')
for key, txt in collection.select(query=q):
    print(key, txt)

1 Text(text='ööbik laulab.')
7 Text(text='öökull ei laula.')


Search using multiple layer attributes:

In [41]:
q = JsonbTextQuery('morph_analysis', lemma='laulma', form='b')
for key, txt in collection.select(query=q):
    print(key, txt)

1 Text(text='ööbik laulab.')


Search using "OR" query:

In [42]:
q = JsonbTextQuery('morph_analysis', lemma='ööbik') | \
    JsonbTextQuery('morph_analysis', lemma='öökull')
for key, txt in collection.select(query=q):
    print(key, txt)

1 Text(text='ööbik laulab.')
7 Text(text='öökull ei laula.')


Search using "AND" query:

In [43]:
q = JsonbTextQuery('morph_analysis', lemma='ööbik') & \
    JsonbTextQuery('morph_analysis', lemma='öökull')
for key, txt in collection.select(query=q):
    print(key, txt)

Search using a composite query:

In [44]:
q = (JsonbTextQuery('morph_analysis', lemma='ööbik') | JsonbTextQuery('morph_analysis', lemma='öökull')) & \
    JsonbTextQuery('morph_analysis', lemma='laulma')
for key, txt in collection.select(query=q):
    print(key, txt)

1 Text(text='ööbik laulab.')
7 Text(text='öökull ei laula.')


or use a convenience method `find_fingerprint`:

In [45]:
for key, txt in collection.find_fingerprint(
                    query={
                        "layer": "morph_analysis",
                        "ambiguous": True,
                        "field": "lemma",
                        "query": [{'ööbik', 'laulma'}, {'öökull', 'laulma'}] # (ööbik AND laulma) OR (öökull AND laulma)
                    },
                    order_by_key=True):
    print(key, txt)

1 Text(text='ööbik laulab.')
7 Text(text='öökull ei laula.')


In [46]:
for key, txt in collection.find_fingerprint(
                    query={
                        "layer": "morph_analysis",
                        "ambiguous": True,
                        "field": "lemma",
                        "query": ['öökull', 'laulma'] # öökull OR laulma
                    },
                    order_by_key=True):
    print(key, txt)

1 Text(text='ööbik laulab.')
7 Text(text='öökull ei laula.')


In [47]:
for key, txt in collection.find_fingerprint(
                    query={
                        "layer": "morph_analysis",
                        "ambiguous": True,
                        "field": "lemma",
                        "query": [{'öökull', 'laulma'}] # öökull AND laulma
                    },
                    order_by_key=True):
    print(key, txt)

7 Text(text='öökull ei laula.')


Delete collection

In [48]:
collection.delete()

INFO:db.py:1054: collection 'my_collection' deleted


## Working with layers

Let's say you want to create a collection which stores only layers up to "sentences":

In [49]:
collection = storage.get_collection('collection_with_layers')
collection.create()

INFO:db.py:84: new empty collection 'collection_with_layers' created


In [50]:
collection.insert(Text('see on esimene lause').tag_layer(["sentences"]))
collection.insert(Text('see on teine lause').tag_layer(["sentences"]));

Check what layers are present:

In [51]:
for key, text in collection.select():
    print(key, text, text.layers.keys())

1 Text(text='see on esimene lause') dict_keys(['tokens', 'sentences', 'compound_tokens', 'words'])
2 Text(text='see on teine lause') dict_keys(['tokens', 'sentences', 'compound_tokens', 'words'])


Now, you want to add new layers "my_first_layer" and "my_second_layer" to store other information. However, you want to store them in a separate table. For this purpose collection object has a `create_layer` method:

In [52]:
from estnltk.storage.postgres import RowMapperRecord


layer1 = "my_first_layer"
layer1_table = collection.layer_name_to_table_name(layer1)
tagger1 = VabamorfTagger(disambiguate=False, layer_name=layer1)

def row_mapper_1(row):
    text_id, text = row[0], row[1]
    layer = tagger1.tag(text, return_layer=True)
    return [RowMapperRecord(layer=layer, meta=None)]

collection.create_layer(layer1, data_iterator=collection.select(), row_mapper=row_mapper_1)


layer2 = "my_second_layer"
layer2_table = collection.layer_name_to_table_name(layer2)
tagger2 = VabamorfTagger(disambiguate=False, layer_name=layer2)

def row_mapper_2(row):
    text_id, text = row[0], row[1]
    layer = tagger2.tag(text, return_layer=True)
    return [RowMapperRecord(layer=layer, meta=None)]

collection.create_layer(layer2, data_iterator=collection.select(), row_mapper=row_mapper_2)

INFO:db.py:684: collection: 'collection_with_layers'
INFO:db.py:695: preparing to create a new layer: 'my_first_layer'
INFO:db.py:755: layer created: 'my_first_layer'
INFO:db.py:684: collection: 'collection_with_layers'
INFO:db.py:695: preparing to create a new layer: 'my_second_layer'
INFO:db.py:755: layer created: 'my_second_layer'


Specified `callable` function is applied to each `text` entry in a collection and is expected to return a `Layer` instance.

If your `tagger` uses layers that are stored in separate tables, use `layers` argument to specify the layers to fetch. These layers will be merged with the text object before passing it to `callable`:

```
collection.create_layer(layer,
                        layers=['layer1', 'layer2', ...],
                        callable=lambda t: tagger.tag(t, return_layer=True))
```

Make sure the new layer has been created:

In [53]:
collection.get_layer_names()

['my_first_layer',
 'tokens',
 'compound_tokens',
 'my_second_layer',
 'sentences',
 'words']

### Searching layers

Iterate over collection and new layers using `select` method:

In [54]:
for key, text in collection.select(layers=['my_first_layer', 'my_second_layer']):
    print(key, text, text.layers.keys())

1 Text(text='see on esimene lause') dict_keys(['my_first_layer', 'tokens', 'compound_tokens', 'my_second_layer', 'sentences', 'words'])
2 Text(text='see on teine lause') dict_keys(['my_first_layer', 'tokens', 'compound_tokens', 'my_second_layer', 'sentences', 'words'])


Notice that the detached layers 'my_first_layer' and 'my_first_layer' are available within the returned `Text` object.

Search layer using `JsonbLayerQuery`:

In [55]:
for key, text in collection.select(layer_query={
        layer1: JsonbLayerQuery(layer_table=layer1_table, lemma='esimene') | \
                JsonbLayerQuery(layer_table=layer1_table, lemma='teine')
        }):
    print(key, text)

1 Text(text='see on esimene lause')
2 Text(text='see on teine lause')


Search over multiple layers using `JsonbLayerQuery`:

In [56]:
for key, text in collection.select(layer_query={
        layer1: JsonbLayerQuery(layer_table=layer1_table, lemma='esimene') | \
                JsonbLayerQuery(layer_table=layer1_table, lemma='teine'),
        layer2: JsonbLayerQuery(layer_table=layer2_table, lemma='esimene')
        }):
    print(key, text)

1 Text(text='see on esimene lause')


The same layer query can be specified using convenience method `find_fingerprint`:

In [57]:
for key, text in collection.find_fingerprint(layer_query={
            layer1: {
                "field": "lemma",
                "query": ["esimene", "teine"],
                "ambiguous": True
            },
            layer2: {
                "field": "lemma",
                "query": ["esimene"],
                "ambiguous": True
            }}):
    print(key, text)

1 Text(text='see on esimene lause')


Delete layer

In [58]:
collection.delete_layer(layer1)

INFO:db.py:1019: layer deleted: 'my_first_layer'


### Indexing layers

Ngram index enables to index ngrams in layer attributes.
For example, a bigram index on an attribute with values `['see', 'on', 'esimene', 'lause']` will contain pairs *'see-on'*, *'on-esimene'*, *'esimene-lause'*.
Indices of a higher order are also supported.

To build an ngram index, provide an argument *ngram_index* when creating a new layer.
The following code creates a bi-gram index on an attribute *lemma* for a newly created layer *indexed_layer*:

In [59]:
indexed_layer = 'indexed_layer'
indexed_layer_table = collection.layer_name_to_table_name(indexed_layer)
tagger1 = VabamorfTagger(disambiguate=False, layer_name=indexed_layer)

def row_mapper(row):
    text_id, text = row[0], row[1]
    layer = tagger1.tag(text, return_layer=True)
    return [RowMapperRecord(layer=layer, meta=None)]

collection.create_layer(indexed_layer,
                        data_iterator=collection.select(),
                        row_mapper=row_mapper_1,
                        ngram_index={"lemma": 2})

INFO:db.py:684: collection: 'collection_with_layers'
INFO:db.py:695: preparing to create a new layer: 'indexed_layer'
INFO:db.py:755: layer created: 'indexed_layer'


To search an ngram index, use method `find_fingerprint` along with `layer_ngram_query` argument.

Search entries containing lemma bigram 'see-olema':

In [60]:
q = {indexed_layer: {
        "lemma": [("see", "olema")]
    }}
for key, text in collection.find_fingerprint(layer_ngram_query=q):
    print(key, text)

1 Text(text='see on esimene lause')
2 Text(text='see on teine lause')


Search 'teine-lause' OR 'olema-esimene':

In [61]:
q = {indexed_layer: {
        "lemma": [("teine", "lause"), ("olema", "esimene")]
    }}
for key, text in collection.find_fingerprint(layer_ngram_query=q):
    print(key, text)

1 Text(text='see on esimene lause')
2 Text(text='see on teine lause')


Search 'see-olema' AND 'olema-esimene':

In [62]:
q = {indexed_layer: {
        "lemma": [[("see", "olema"), ("olema", "esimene")]]
    }}
for key, text in collection.find_fingerprint(layer_ngram_query=q):
    print(key, text)

1 Text(text='see on esimene lause')


Delete schema and all collections

In [63]:
delete_schema(storage)

Close database connection

In [64]:
storage.close()

## Working with fragments

In [65]:
schema = "test_fragment"
storage = PostgresStorage(pgpass_file='~/.pgpass',
                               schema=schema, dbname='test_db')
create_schema(storage)

INFO:postgres_storage.py:97: connecting to host: 'localhost', port: '5432', dbname: 'test_db', user: 'pault'
INFO:postgres_storage.py:109: role: 'pault'


In [ ]:
table_name = 'fragment_demo'

col = storage.get_collection(table_name)
col.create()

layer_fragment_name = "layer_fragment_1"

col.create_layer(layer_fragment_name,
                 data_iterator=col.select(),
                 row_mapper=None)

fragment_name = "fragment_1"

def row_mapper(row):
    text_id, text = row[0], row[1]
    fragments = [text[layer_fragment_name],
                 text[layer_fragment_name]]
    return fragments

col.create_fragment(fragment_name,
                    data_iterator=col.select_raw(layers=[layer_fragment_name]),
                    row_mapper=row_mapper,
                    create_index=False,
                    ngram_index=None)

assert storage.table_exists(col.fragment_name_to_table_name(fragment_name))
assert col.has_fragment(fragment_name)
assert fragment_name in col.get_fragment_names()

col.delete_fragment(fragment_name)

assert self.storage.table_exists(col.fragment_name_to_table_name(fragment_name))
assert col.has_fragment(fragment_name)
assert fragment_name in col.get_fragment_names()

col.delete()

In [ ]:
table_name = 'fragment_test'
col = storage.get_collection(table_name)
col.create()

text1 = Text('see on esimene lause').tag_layer(["sentences"])
col.insert(text1)
text2 = Text('see on teine lause').tag_layer(["sentences"])
col.insert(text2)

layer_fragment_name = "layer_fragment_1"
tagger = VabamorfTagger(disambiguate=False, layer_name=layer_fragment_name)
col.create_layer(layer_fragment_name,
                 data_iterator=col.select(),
                 row_mapper=lambda row: [RowMapperRecord(layer=tagger.tag(row[1], return_layer=True),
                                                         meta=None)])

assert col.has_layer(layer_fragment_name)

fragment_name = "fragment_1"

def row_mapper(row):
    text_id, text, parent_id, parent_layer = row[0], row[1], row[2], row[3]
    fragments = [RowMapperRecord(layer=parent_layer[layer_fragment_name], meta=None),
                 RowMapperRecord(layer=parent_layer[layer_fragment_name], meta=None)]
    return fragments

col.create_fragment(fragment_name,
                    data_iterator=col.select_raw(layers=[layer_fragment_name]),
                    row_mapper=row_mapper,
                    create_index=False,
                    ngram_index=None)

rows = list(col.select_fragment_raw(fragment_name, layer_fragment_name))
assert len(rows) == 4

row = rows[0]
self.assertEqual(len(row), 6)
self.assertIsInstance(row[0], int)
self.assertIsInstance(row[1], Text)
self.assertIsInstance(row[2], int)
self.assertIsInstance(row[3], Layer)
self.assertIsInstance(row[4], int)
self.assertIsInstance(row[5], Layer)

In [66]:
delete_schema(storage)
storage.close()